## 💻 분석 데이터 : 버스 승차요청 이력 데이터  
-> 사용자의 요청에 따라 운행 경로를 바꿔 운행하는 버스 서비스

### (1) 전처리 진행

### - 데이터 불러오기

In [34]:
import pandas as pd
data=pd.read_csv("new_dataset.csv", low_memory=False)

In [47]:
df=data

In [48]:
#전체 열 보이도록 수정
pd.set_option('display.max_columns', None)

In [55]:
#df.info()
#column개수=41, row개수 : 2,064,519

### - 컬럼별 uniuqe값 확인

In [50]:
df.호출유형.unique() #I
df.지역.unique() #계양, 검단, 송도
df.배차차량.unique() #nan, '계양1호차 ', '검단1호차 ', '송도1호차 ', '송도6호차 ', '송도2호차 ', '계양2호차 ', '검단2호차 ', '검단4호차 ', '검단3호차 ', '송도7호차 ', '송도8호차 ', '송도5호차 ', '송도3호차 ', '송도4호차 '
df.상태.unique() #'시뮬레이션취소', '탑승취소', '배차불가', '하차완료', '시스템취소', '배차불가 (중복된 이용신청)','배차불가 (운행가능차량없음)'
df.요청시간.min(), df.요청시간.max() #'2021-07-19 05:31:52', '2022-12-17 22:59:47'

('2021-07-19 05:31:52', '2022-12-17 22:59:47')

### - 컬럼 삭제

In [51]:
#df=df.drop(columns=['']) --삭제할 컬럼 지정

#분석에 필요하지 않은 컬럼 삭제
df=df.drop(columns=['순번', '위도', '경도', '승무사원명'])
df=df.drop(columns=['취소시간', '승차지시뮬레이션시간', '승차지최초도착예정시간', '승차지도착예정시간', '하차지시뮬레이션시간', '하차지최초도착예정시간', '하차지도착예정시간', '배차확정시간', '요청시간차이', '속도'])

#결제정보/평가정보 컬럼은 분석에 필요하지않으므로 삭제
df=df.drop(columns=['배차요청위치', '평가점수_빠른이동', '평가점수_안전운전', '평가점수_버스청결도','평가메모', '평가점수_총점'])

#요금 컬럼은 분석에 필요하지않으므로 삭제
df=df.drop(columns='요금')

#호출유형은 I 단일값이므로 삭제
df=df.drop(columns=['호출유형'])

#승차시간, 도착시간을 기준으로 분석진행
df=df.drop(columns=['승차지도착시간', '하차지도착시간'])

#요청식별id가 존재하므로 탑승권id, 사용자id 삭제
#분석에서 의미가 없다고 판단되는 구독제_여부, 이용신청id삭제
df=df.drop(columns=['탑승권아이디', '사용자id', '구독제 여부', '구독제_여부', '이용신청id'])

#배차차량과 배차차량번호가 일치하므로 배차차량 기준으로 분석진행
df=df.drop(columns='배차차량번호')

### - 데이터 파악

In [52]:
#상태컬럼이 하차완료일때 배차차량컬럼의 unique값 확인
df[df['상태']=='하차완료']['배차차량'].unique()

array(['송도6호차 ', '송도2호차 ', '계양1호차 ', '검단2호차 ', '검단4호차 ', '송도7호차 ',
       '송도8호차 ', '송도5호차 ', '검단1호차 ', '송도4호차 ', '송도3호차 ', '송도1호차 ',
       '검단3호차 ', '계양2호차 '], dtype=object)

In [53]:
#상태컬럼이 하차완료일때 배차차량컬럼의 null값 확인
df[df['상태']=='하차완료']['배차차량'].isna().sum()

0

-> 상태컬럼이 '하차완료'인 경우에는 배차차량이 null값 없이 다 지정되어 있음을 확인할 수 있음.

### - 분석을 위한 컬럼 생성

- groupby요청식별id 요청시간.min()인 것을 최초요청시간 컬럼에 추가

In [54]:
#연산을 위한 컬럼만 저장
연산=df[['요청시간', '요청식별id']]

#groupby 함수를 사용해 요청식별id별 최초 요청시간 계산
최초요청시간=연산.groupby('요청식별id').min().reset_index()
최초요청시간.columns=['요청식별id', '최초요청시간']
최초요청시간.head(5)

요청식별id               최초요청시간
0       0  2022-05-03 14:47:56
1       1  2022-05-03 14:48:44
2       2  2022-08-07 23:54:51
3       3  2022-10-21 21:40:34
4       4  2022-10-22 08:21:29

In [57]:
#원래 데이터프레임과 요청식별id를 기준으로 병합
df2=pd.merge(df, 최초요청시간)

In [60]:
#최초요청시간 컬럼이 추가된 것을 확인
df2.head(1)

지역 배차차량       상태                 요청시간          승차정류장명 승차정류장방면  \
0  계양  NaN  시뮬레이션취소  2022-05-03 14:47:56  계양역(계양중학교 방면)    귤현역방면   

          하차정류장명     하차정류장방면 승차시간 하차시간  탑승총계  어른  청소년  어린이   거리 회원유형  승차정류장id  \
0  목상동솔밭.이주단지입구   목상동마을입구 방면  NaN  NaN     1   1    0    0  0.0   어른  41659.0   

   하차정류장id  요청식별id  누적요청시간  누적요청횟수  실수요여부               최초요청시간  
0  41316.0       0     0.0       1      1  2022-05-03 14:47:56

### - 전처리 파일 저장

In [61]:
df2.to_csv("new_전처리.csv", index=False)